# 1. 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd

In [2]:
import pickle

In [42]:
service = Service('chromedriver.exe')
driver = webdriver.Chrome(service = service)

In [4]:
isbn13 = pd.read_pickle('isbn13.pkl')

In [194]:
isbn13

0        9791190710343
1        9791190710350
2        9791190710367
3        9791128858338
4        9788954685603
             ...      
54971    9791185020136
54972    9788997838127
54973    9788956591858
54974    9788987794938
54975    9788995696972
Name: isbn13, Length: 54976, dtype: object

In [ ]:
rank_mean = [] # 도서 평균 평점
rank_percentage = [] # 평균 percentage
review_title = [] # 리뷰 제목
review_id = [] # 리뷰 id
review_rank = [] # 리뷰 평점
review_text = [] # 리뷰 
isbn_idx = []


for isbn in isbn13[1171:2000]: # 페이지 입력
    url = "http://www.yes24.com/product/search?query={}".format(isbn)
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 페이지에 있는 도서 링크(50개)
    book_links = soup.select('a.gd_name')
    goods = []
    for link in book_links:
        a= link['href'].split('/')[3] 
        goods.append(a)
    for item in goods:
        del(goods[1:])
        driver.get('http://www.yes24.com'+'/Product/Goods/{}'.format(item)) # 상품 페이지
        time.sleep(1)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        try:
            mean = soup.select('div.gd_rating > em.yes_b')[0].text
            rank_mean.append([mean]) # 도서 평균 평점
        except:
            mean = []
            rank_mean.append(mean) # 도서 평균 평점
        rank_p = []
        try:
            for k in range(5):
                rank_ = soup.select('span.barGraphBase')[k].text # 평점 percentage
                rank_p.append(rank_)
            rank_percentage.append(rank_p)
        except:
            rank_ = []
            rank_percentage.append(rank_)
        try:
            review_t = []  # 각각의 책의 리뷰제목 리스트
            review_i = []
            review_r = []
            review_te = []
            for page in range(1,3): # 리뷰 페이지 2개
                driver.get('http://www.yes24.com/Product/communityModules/GoodsReviewList/{}?Type=ALL&_=1648001707174&Sort=2&PageNumber={}'.format(item, page))
                time.sleep(1)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                
                blog_link = []
                for i in range(5): # 리뷰 한 페이지에 있는 5개 리뷰 가져오기
                    try:
                        title = soup.select('span.review_tit')[i].text.strip() # 리뷰 제목
                        review_t.append(title)
                        r_id = soup.select('a.lnk_id')[i].text # 리뷰 id
                        review_i.append(r_id)
                        rank = soup.select('span.review_rating')[i].text.split('\n')[1][-2:] # 리뷰 평점
                        review_r.append(rank)
                        blog_l = soup.select('div.review_lnk a')[i]['href'] # 리뷰 블로그 링크
                        blog_link.append(blog_l)
                    except:
                        pass
                for j in range(5): # 리뷰 한 페이지에 있는 5개 블로그 내용 가져오기
                        driver.get(blog_link[j]) # 리뷰 블로그 링크 들어가기
                        time.sleep(1)
                        html = driver.page_source
                        soup = BeautifulSoup(html, 'html.parser')
                        text = soup.select('div.blogContArea')[0].text # 리뷰 내용 
                        review_te.append(text)
        except:
            pass
        review_title.append(review_t)
        review_id.append(review_i)
        review_rank.append(review_r)
        review_text.append(review_te)
        isbn_idx.append(isbn)

In [11]:
col = ['isbn','rank_mean','rank_percentage','review_title','review_id','review_rank','review_text']
user = pd.DataFrame([isbn_idx,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text]).T
user.columns = col
user

,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text
0,[9.4],"[94%, 0%, 0%, 0%, 6%]","[사실은 “나 자신을 위한” 선택, 지구를 위해 모두가 채식할 수는 없지만 (하루...","[c******0, 고*맘, r*****y, 강*제, w********6, s***...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]",[\n채식주의자한강 작가의 채식주의자를 읽을 때나는 그 조용한 잔인함에 매우 많이 ...
1,[10.0],[],[],[],[],[]
2,[9.8],"[86%, 14%, 0%, 0%, 0%]","[오늘 당장 꿈을 향해 화살을 쏴라, 불과 두줄로 삶을 돌아볼 수 있는 기회, 두줄...","[h*******1, l******2, d******e, 같*가, 물*기, 요*한,...","[5점, 5점, 5점, 5점, 5점, 5점, 4점]","[\n광화문에 내걸린 '겸손은 머리의 각도가 아니라, 마음의 각도다'란 문구를 만드..."
3,[9.7],"[93%, 7%, 0%, 0%, 0%]","[슬기로운 검사생활, ﻿슬기로운 검사생활, 슬기로운 검사생활, [슬기로운 검사생활]...","[책*물, 매*얼, k*******2, 도*비, c********5, 명******...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]",[\nㅍ\n \n \n \n \n\n \n \n \n사건이라 쓰고 사연이라 읽는 것...
4,[],[],[],[],[],[]
5,[8.0],[],[],[],[],[]
6,[],[],[],[],[],[]
7,[8.0],"[0%, 100%, 0%, 0%, 0%]","[도서관으로 가출한 사서, 일상 에세이]",[k*******5],[4점],[\n\n \n삶의 기억 중 많은 부분이 도서관을 배경으로 하고 있다. 어쩌면 그럴...
8,[8.0],"[0%, 100%, 0%, 0%, 0%]",[때로는 마음도 체한다],[조*배],[4점],[\n『 때로는 마음도 체한다 』\n\n저자: 변상규\n출판: 에디터 (2014년 ...
9,[9.0],"[50%, 50%, 0%, 0%, 0%]","[나의 작은 날들에게, 이름 지어 주고 싶은 날들이 있다]","[g********9, 하**늘]","[5점, 4점]","[\n곱게 책꽂이에 넣어둔 일기장을 꺼낸 것 같다.가만히 숨겨뒀던 그날의 날씨, 옷..."


In [13]:
user.to_pickle('user_1171-1170.pkl', protocol = 4)

In [34]:
isbn13[1150] # 25 >> 25, 27 >> 29

'9788954621496'

In [38]:
user1 = pd.read_pickle('user_1000-1123.pkl')
user2 = pd.read_pickle('user_1124-1170.pkl') # 이전코드

In [39]:
pd.concat([user1, user2])

,rank_mean,rank_percentage,review_title,review_id,review_rank,review_text
0,[9.6],"[75%, 25%, 0%, 0%, 0%]","[지금 잘 살아야 하는 이유에 대한 통찰 [나는 환생을 믿지 않았다], 명료함을 불...","[하*애, s****a, j**a, H************2]","[5점, 4점, 5점, 5점]","[\n이상민, 조우종이 진행하는 <차트를 달리는 남자>, 전 세계 별난 이야기들만 ..."
1,[10.0],"[100%, 0%, 0%, 0%, 0%]",[[약삭빠르게 온천] 고독한 미식가의 도심 온천 여행기],[키*],[5점],[\n\n \n드라마로 유명한 일본 만화 <고독한 미식가>의 원작자 구스미 마사유키...
2,[9.8],"[94%, 6%, 0%, 0%, 0%]","[고독함과 마주 할 때에 어떻게 할까?/피카, 세상에 나 혼자라고 느껴질 때, 세상...","[나*이, 보**쥬, 러***니, 샤*라, 폭**상, 카*라, 줄**북, 설*산, ...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]",[\n이 책은?\n \n스스로 고독과 마주친 사람의 고독에 대한 대면을 그리고 있...
3,[9.8],"[92%, 8%, 0%, 0%, 0%]","[세상 끝에서 춤추다, 『세상 끝에서 춤추다』 르 귄을 만든 시간의 기록, 르귄의 ...","[모*, 누*, j*********n, 키*, 깔**s, y******k, o***...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점]",[\n\n \n도 서: 세상 끝에서 춤추다\n \n저 자: 어슐러 K.르 귄\n \...
4,[8.6],"[50%, 44%, 0%, 6%, 0%]","[나의 미래를 예측하는 유일한 방법은 나의 오늘을 보는 것이다, ""데뷔의 순간"" 후...","[s*******r, e*****t, c*****1, r******2, 알***투,...","[5점, 5점, 5점, 5점, 5점, 5점, 5점, 5점, 4점, 4점]","[\n한때 영화계를 기웃거렸던 적이 있다. 대학 시절, 연출자가 되어 보겠다고 시나..."
...,...,...,...,...,...,...
42,[9.5],"[86%, 12%, 2%, 0%, 0%]","[[나도 모르게 생각한 생각들] 2021_095, 나도 모르게_017 (나도 모르게...","[사*님, J*y, 책****벤, n********d, 짓*, i*****6, 긴*...","[4점, 4점, 4점, 5점, 5점, 5점, 5점, 3점, 5점, 5점]",[\n2021_095\n \n\n\n읽은날 : 2021.12.23~2021.12.2...
43,[8.6],"[46%, 46%, 9%, 0%, 0%]","[이렇게 귀여운 할매를 보았나!, 사는게 뭐라고 - 울엄마는 지겹다고 하신다 요즘 ...","[박*리, m**a, 콩*, 부***주, j***7, 오***객, 김*진, 껌***...","[5점, 5점, 5점, 4점, 5점, 4점, 4점, 4점, 5점, 5점]",[\n책 제목을 보고 맥주 광고를 떠 올렸다. 딱히 그 배우를 좋아하는 것은 아닌데...
44,[9.0],"[79%, 7%, 14%, 0%, 0%]","[서평보다 먼저 빈대떡을 부쳤다, 역시 띵시리즈, [맛] 엄마 박완서의 부엌, 엄마...","[t*****1, S***e, 싱*, 책****벤, 숲*래, n**t, 보*비, 잔...","[5점, 5점, 5점, 5점, 3점, 5점, 5점, 5점, 5점, 5점]",[\n고등학교에는 0교시가 있었다. 교실 맨 뒷자리에 앉았던 나는 대부분이 비몽사몽...
45,[10.0],[],[],[],[],[]
